In [1]:
import os
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
import cv2


from tqdm import tqdm
from enum import IntEnum
from typing import Tuple
from IPython.display import Video

# Susceptible–Infected–Removed model

This model is simple simulation of pandemic. In this simulation we have three type of people: susceptible - it's person who can be infected, infected and removed, who represent dead or cured unit.

Infected person can infect other susceptible people in neighborhood radius with some probability (in code it calls probability of contagion). In every move every infected unit can be change our state to removed with some probability (in code it calls probability of death).

In simulation every unit make Brownian motion (https://en.wikipedia.org/wiki/Brownian_motion) in every step. 

In [2]:
class State(IntEnum):
    SUSPECTIBLE = 0
    INFECTED = 1
    REMOVED = 2

    
class Population:
    
    def __init__(self, n: int, size_map: int, neighborhood_radius: int, 
                 propability_of_death: float, propability_of_contation: float):
        self.n = n
        self.size_map = size_map
        self.neighborhood_radius = neighborhood_radius
        self.propability_of_death = propability_of_death
        self.propability_of_contagion= propability_of_contation
        
        self.population = np.zeros((self.n), dtype=[('x', 'f8'), ('y', 'f8'), ('state', 'i4')])
        self.population['x'] = np.random.random(self.n)*self.size_map
        self.population['y'] = np.random.random(self.n)*self.size_map
        self.population = np.sort(self.population, kind='mergesort', order='x')
        
    def gaussian_move(self):
        self.population['x'] += np.random.randn(self.n)
        self.population['y'] += np.random.randn(self.n) 
        
        fmin = np.vectorize(lambda x: min(x, float(self.size_map)))
        fmax = np.vectorize(lambda x: max(x, 0.))
        
        self.population['x'] = fmin(self.population['x'])
        self.population['x'] = fmax(self.population['x'])
        self.population['y'] = fmin(self.population['y'])
        self.population['y'] = fmax(self.population['y'])
        
        self.population = np.sort(self.population, kind='mergesort', order='x')
        
    def seed_infected_people(self, n_infected):
        indexes = np.random.choice(self.n, n_infected)
        self.population['state'][indexes] = State.INFECTED
       
    def remove(self):
        for man in self.population:
            if man['state'] == State.INFECTED and np.random.rand() < self.propability_of_death:
                man['state'] = State.REMOVED
    
    def infect(self):
        @nb.njit
        def numba_loop(table, infected_state, radius, propability_of_contagion):
            def norm(x1, y1, x2, y2):
                return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
            
            for index, man in enumerate(table):
                if man['state'] == infected_state:

                    i = index + 1
                    while(i < len(table) and 
                          table['x'][i] < man['x'] + radius):
                        if(norm(table['x'][i], table['y'][i], man['x'], man['y']) < radius 
                           and np.random.rand() < propability_of_contagion):
                            table[i]['state'] = infected_state
                        i += 1

                    i = index - 1
                    while(i >= 0 and 
                          table['x'][i] > man['x'] - radius):
                        if(norm(table['x'][i], table['y'][i], man['x'], man['y']) < radius
                           and np.random.rand() < propability_of_contagion):
                            table['state'][i] = infected_state
                        i -= 1
                        
        numba_loop(self.population, State.INFECTED, self.neighborhood_radius, self.propability_of_contagion)
                     
            
class Simulation():

    def __init__(self, population: Population):
        self.population = population
        
    def make_step_in_simulation(self):
        self.population.gaussian_move()
        self.population.infect()
        self.population.remove()
        
    def get_SIR_group(self) -> Tuple[np.array, np.array, np.array]:
        suspectible = self.population.population[ \
                          self.population.population['state'] == State.SUSPECTIBLE]
        infected = self.population.population[ \
                          self.population.population['state'] == State.INFECTED]
        removed = self.population.population[ \
                          self.population.population['state'] == State.REMOVED]
        return suspectible, infected, removed

## Making animation of pandemic

In simulation blue dot reprezent suspectible unit, red - infected unit and black removed unit.

In [3]:

class VideoMaker():
    
    def __init__(self, simulation: Simulation):
        self.simulation = simulation
        
    def _make_step_image(self):
        suspectible, infected, removed = self.simulation.get_SIR_group()
        
        plt.figure(figsize=(16, 16), dpi=80)
        plt.scatter(suspectible['x'], suspectible['y'], c='b')
        plt.scatter(infected['x'], infected['y'], c='r')
        plt.scatter(removed['x'], removed['y'], c='black')
        plt.axis('off')
        plt.savefig('pandemic_step.png', bbox_inches='tight')
        plt.close()
        
    def _get_next_image(self) -> np.array:
        self._make_step_image()
        image = cv2.imread('pandemic_step.png')
        image = cv2.resize(image, (1000, 1000)) 
        return image
        
    def make_movie(self, step: int):
        image = self._get_next_image()
        width, height, _ = image.shape
                
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        movie = cv2.VideoWriter('simulation.mp4', fourcc, 16.0, (width, height))
        
        for _ in tqdm(range(step), desc="Making video in progress:"):
            movie.write(image)
            
            if self.simulation.make_step_in_simulation() == 'Done':
                break
            else:
                image = self._get_next_image()
        movie.release()


## Let's run it together

In [4]:
population = Population(3_000, 100, 1, 0.10, 0.9)
population.seed_infected_people(1)

simulation = Simulation(population)

video_maker = VideoMaker(simulation)
video_maker.make_movie(100)

Making video in progress:: 100%|██████████| 100/100 [01:22<00:00,  1.21it/s]
